In [1]:
%pip install stanza


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install politenessr
%pip install pytorch_pretrained_bert 
%pip install pytorch_transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import json
import requests
import nltk
import pandas as pd
import pickle
import stanza as stanza

from nltk.tokenize.punkt import PunktSentenceTokenizer

from tqdm import tqdm
from politenessr import Politenessr

from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [11]:
stanza.download('en')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
def load_dataset():
    file_train = "/kaggle/input/diplomacy/test.jsonl"
    file_test = "/kaggle/input/diplomacy/train.jsonl"
    file_validation = "/kaggle/input/diplomacy/validation.jsonl"

    # Open and read the files locally
    with open(file_train, 'r') as f_train:
        deception_train = [json.loads(line) for line in f_train]

    with open(file_test, 'r') as f_test:
        deception_test = [json.loads(line) for line in f_test]

    with open(file_validation, 'r') as f_validation:
        deception_validation = [json.loads(line) for line in f_validation]

    # Merge the datasets
    deception = []
    deception.extend(deception_train)
    deception.extend(deception_validation)
    deception.extend(deception_test)
    
    return deception


In [13]:
deception = load_dataset()
print(len(deception))
with open('deception_without_politeness_sentiment.pkl', 'wb') as f:
    pickle.dump(deception, f)

252


In [14]:
def flatten_deception(dataset):
    flat_data = {}
    for game in dataset:
        game_length = len(game["messages"])
        for k, v in game.items():
            if k == "game_id":
                v = [v] * game_length
            if k == "players":
                v = [",".join(v)] * game_length
                    
            if k not in flat_data:
                flat_data[k] = v
            else:
                flat_data[k].extend(v)
                
    return flat_data

df = pd.DataFrame.from_dict(flatten_deception(deception))

In [15]:
df.shape


(17289, 13)

In [16]:
pr = Politenessr()
politeness = pr.predict(list(df['messages']))
df['politeness'] = politeness

423140KB [00:04, 91070.37KB/s]                            
/usr/local/lib/python3.10/dist-packages/politenessr/politenessr.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

In [17]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')


negative_sentiments = []
neutral_sentiments = []
positive_sentiments = []
    
for message in tqdm(df['messages']):
    counts = [0, 0, 0]
    message = nlp(message)
    if len(message.sentences) > 0:
        for sentence in message.sentences:
            counts[sentence.sentiment] += 1
            normalized_counts = [count / len(message.sentences) for count in counts]
    else:
        normalized_counts = [0, 0, 0]

    
    
    negative_sentiments.append(normalized_counts[0])
    neutral_sentiments.append(normalized_counts[1])
    positive_sentiments.append(normalized_counts[2])
            
df["negative_sentiment"] = negative_sentiments
df["neutral_sentiment"] = neutral_sentiments
df["positive_sentiment"] = positive_sentiments

100%|██████████| 17289/17289 [05:54<00:00, 48.75it/s]


In [18]:
nltk.download("vader_lexicon")

sid = SentimentIntensityAnalyzer()
vader_score = []
for message in tqdm(df['messages']):
    vader_score.append(sid.polarity_scores(message)["compound"])

df["vader_score"] = vader_score

/usr/local/lib/python3.10/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


100%|██████████| 17289/17289 [00:03<00:00, 4839.19it/s]


In [19]:
df.sample(5)


,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
4825,Huh. I guess France is lying to me then?,True,True,england,italy,1196,105,Spring,1905,5,-2,"italy,england",1,3.004755,0.00,1.00,0.0,-0.5267
11865,Also kinda got on my high horse with what you ...,False,True,england,turkey,1301,91,Spring,1905,6,1,"england,turkey",3,3.399129,0.75,0.25,0.0,0.5454
9764,Germany is talking to me about heading back,True,True,england,austria,2676,96,Spring,1905,6,-2,"austria,england",2,3.250013,0.00,1.00,0.0,0.0000
10817,Have you heard anything from Turkey?,True,True,germany,austria,260,7,Fall,1901,3,0,"germany,austria",3,3.197506,0.00,1.00,0.0,0.0000
4594,Well that is damn annoying. I’m surprised he t...,False,True,italy,austria,1750,116,Winter,1906,10,4,"italy,austria",1,2.882527,0.50,0.50,0.0,-0.3400


In [20]:
df.to_pickle("deception_df.pkl")